In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# CSV 파일 읽기
X = pd.read_csv('행정안전부_지역별(행정동) 성별 연령별 주민등록 인구수_20240630.csv', encoding='cp949')
X2 = pd.read_csv('20240803013511_광역별 방문자 수.csv', encoding='cp949')

# 불필요한 열 삭제
data_df = X.iloc[:, [3, 5]]  # 시군구명, 인구수 열
data_df2 = X2.iloc[:, [1, 2]]  # 시군구명, 방문자수 열
data_df.columns = ['시군구명', '인구수']
data_df2.columns = ['시군구명', '방문자수']

# '시군구명' 열의 데이터에서 공백 뒤의 내용 제거
data_df['시군구명'] = data_df['시군구명'].str.split().str[0]
data_df2['시군구명'] = data_df2['시군구명'].str.split().str[0]

# 빈 값을 '세종특별자치시'로 대체
data_df['시군구명'] = data_df['시군구명'].str.strip().replace('', '세종특별자치시')
data_df2['시군구명'] = data_df2['시군구명'].str.strip().replace('', '세종특별자치시')

# 데이터 그룹화
data_df_grouped = data_df.groupby('시군구명', as_index=False).agg({'인구수': 'sum'})
data_df2_grouped = data_df2.groupby('시군구명', as_index=False).agg({'방문자수': 'sum'})

# 데이터 병합
merged_data = pd.merge(data_df_grouped, data_df2_grouped, on='시군구명')

# 관광객 밀도 계산
merged_data['관광객 밀도'] = merged_data['방문자수'] / merged_data['인구수']
 
# 데이터 정규화
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(merged_data[['관광객 밀도']])
scaled_df = pd.DataFrame(scaled_data, columns=['관광객 밀도'], index=merged_data['시군구명'])

# 한국 시군구 경계 데이터 불러오기
korea_sgg = gpd.read_file('./sig.shp', encoding='cp949')

# 'SIG_KOR_NM'에 대한 인덱스를 설정하여 병합
scaled_df.reset_index(inplace=True)  # 인덱스를 리셋하여 '시군구명' 열을 일반 열로 만듭니다.
scaled_df.rename(columns={'시군구명': 'SIG_KOR_NM'}, inplace=True)  # 열 이름 변경

# 경계 데이터에서 'SIG_KOR_NM'과 정규화된 데이터를 병합
merged_map = korea_sgg.merge(scaled_df, on='SIG_KOR_NM', how='inner')

# 관광객 밀도에 따라 색상 매핑
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
merged_map.boundary.plot(ax=ax, linewidth=1)  # 경계선 그리기
merged_map.plot(column='관광객 밀도', ax=ax, legend=True,
                legend_kwds={'label': "관광객 밀도",
                             'orientation': "horizontal"},
                cmap='Reds', missing_kwds={"color": "lightgrey", "label": "없음"})
# 배경 색 설정
plt.gca().set_facecolor('lightblue')  # 배경 색을 연한 파란색으로 설정

# 제목 설정
plt.title('한국 지역 거주민 대비 관광객 밀도 히트맵')
plt.show()